In [ ]:
import zipfile
with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('./dataset')

In [1]:
import os
import cv2
import shutil
from shutil import copyfile
import numpy as np
import pandas as pd

In [2]:
os.mkdir('dataset/yolo/')
os.mkdir('dataset/yolo/val/')
os.mkdir('dataset/yolo/train/')
os.mkdir('dataset/yolo/val/images/')
os.mkdir('dataset/yolo/val/labels/')
os.mkdir('dataset/yolo/train/images/')
os.mkdir('dataset/yolo/train/labels/')

In [3]:
orig_train = 'dataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
orig_val = 'dataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/test'

training_images = 'dataset/yolo/train/images/'
training_labels = 'dataset/yolo/train/labels/'
validation_images = 'dataset/yolo/val/images/'
validation_labels = 'dataset/yolo/val/labels/'

In [4]:
data = pd.read_csv(r'dataset/Chest_xray_Corona_Metadata.csv')
data

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN
...,...,...,...,...,...,...
5905,5928,person1637_virus_2834.jpeg,Pnemonia,TEST,NaN,Virus
5906,5929,person1635_virus_2831.jpeg,Pnemonia,TEST,NaN,Virus
5907,5930,person1634_virus_2830.jpeg,Pnemonia,TEST,NaN,Virus
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,NaN,Virus


In [5]:
image_names = [x for x in data['X_ray_image_name']]
dataset_type = [x for x in data['Dataset_type']]
healthy_or_not = [x for x in data['Label']]
virus_or_not = [x for x in data['Label_1_Virus_category']]

print(len(image_names))
print(len(dataset_type))
print(len(healthy_or_not))
print(len(virus_or_not))

5910
5910
5910
5910


In [27]:
def preprocess_image(img_path, save_path):
    img = cv2.imread(img_path, 0) #cv2.IMREAD_COLOR
#     resultimage = np.zeros((800, 800))
    norm_img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
#     equ_img = cv.equalizeHist(norm_img)
    # create a CLAHE object (Arguments are optional).
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    equ_img = clahe.apply(norm_img)
    cv2.imwrite(save_path, equ_img)

In [28]:
normal = 0
virus = 0
bacteria = 0

for index, image in enumerate(image_names):
    if healthy_or_not[index] == 'Normal':
        normal += 1
        if dataset_type[index] == 'TRAIN':
#             copyfile(os.path.join(orig_train, image), os.path.join(training_images, image))
            preprocess_image(os.path.join(orig_train, image), os.path.join(training_images, image))
            name = image.split('.')[0]
            boxes_file = open(os.path.join(training_labels, name+'.txt'), 'w')
#             img = cv2.imread(os.path.join(orig_train, image))
#             height, width, _ = img.shape
#             boxes_file.write('0 %f %f %f %f\n' %(width//2, height//2, width//2, height//2))
            boxes_file.write('0 0.5 0.5 1.0 1.0')
            boxes_file.close()
            
        elif dataset_type[index] == 'TEST':
#             copyfile(os.path.join(orig_val, image), os.path.join(validation_images, image))
            preprocess_image(os.path.join(orig_val, image), os.path.join(validation_images, image))
            name = image.split('.')[0]
            boxes_file = open(os.path.join(validation_labels, name+'.txt'), 'w')
            boxes_file.write('0 0.5, 0.5 1.0 1.0')
            boxes_file.close()
            
    elif healthy_or_not[index] == 'Pnemonia':
        if virus_or_not[index] == 'Virus':
            virus += 1
            if dataset_type[index] == 'TRAIN':
#                 copyfile(os.path.join(orig_train, image), os.path.join(training_images, image))
                preprocess_image(os.path.join(orig_train, image), os.path.join(training_images, image))
                name = image.split('.')[0]
                boxes_file = open(os.path.join(training_labels, name+'.txt'), 'w')
                boxes_file.write('1 0.5, 0.5 1.0 1.0')
                boxes_file.close()

            elif dataset_type[index] == 'TEST':    
#                 copyfile(os.path.join(orig_val, image), os.path.join(validation_images, image))
                preprocess_image(os.path.join(orig_val, image), os.path.join(validation_images, image))
                name = image.split('.')[0]
                boxes_file = open(os.path.join(validation_labels, name+'.txt'), 'w')
                boxes_file.write('1 0.5, 0.5 1.0 1.0')
                boxes_file.close()
        else:
            bacteria += 1
            if dataset_type[index] == 'TRAIN':
#                 copyfile(os.path.join(orig_train, image), os.path.join(training_images, image))
                preprocess_image(os.path.join(orig_train, image), os.path.join(training_images, image))
                name = image.split('.')[0]
                boxes_file = open(os.path.join(training_labels, name+'.txt'), 'w')
                boxes_file.write('2 0.5, 0.5 1.0 1.0')
                boxes_file.close()

            elif dataset_type[index] == 'TEST':    
#                 copyfile(os.path.join(orig_val, image), os.path.join(validation_images, image))
                preprocess_image(os.path.join(orig_val, image), os.path.join(validation_images, image))
                name = image.split('.')[0]
                boxes_file = open(os.path.join(validation_labels, name+'.txt'), 'w')
                boxes_file.write('2 0.5, 0.5 1.0 1.0')
                boxes_file.close()
                
print(normal, virus, bacteria)

1576 1555 2779
